<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit (Python)</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text from PDF and convert them into parquet files
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Exact Dedup**: Chunks with exact same content are filtered out
- **Doc_ID generation**: Each chunk is assigned a uniq id, based on content and hash
- **Fuzzy Dedup**: Eliminate chunks that are 'very similar' content
- **Doc quality**: Scores the documents based on criteria like number of words, if it contains bad words ..etc
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

In [1]:
import os 

## Configuration
class MyConfig:
    pass 

MY_CONFIG = MyConfig ()

## Input Data - configure this to the folder we want to process
MY_CONFIG.INPUT_DATA_DIR = "input"
MY_CONFIG.OUTPUT_FOLDER = "output"
MY_CONFIG.OUTPUT_FOLDER_FINAL = os.path.join(MY_CONFIG.OUTPUT_FOLDER , "output_final")

## Embedding model
MY_CONFIG.EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'

## RAY CONFIGURATION
num_cpus_available =  os.cpu_count()
# print (num_cpus_available)
# MY_CONFIG.RAY_NUM_CPUS = num_cpus_available // 2  ## use half the available cores for processing
MY_CONFIG.RAY_NUM_CPUS =  1
# print (MY_CONFIG.RAY_NUM_CPUS)
MY_CONFIG.RAY_MEMORY_GB = 2  # GB
# MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
MY_CONFIG.RAY_RUNTIME_WORKERS = 2

### Download Data

We will use [Walmart annual report PDFs](https://github.com/sujee/data/tree/main/data-prep-kit/walmart-reports-1) as our input data.

Feel free to substitute your data

In [2]:
import os, sys
import shutil
from utils import download_file

## Download the data files
shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart-10K-Reports-Optimized_2023.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart-10K-Reports-Optimized_2023.pdf' ))

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart_2024.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart_2024.pdf' ))

download_file (url = 'https://raw.githubusercontent.com/sujee/data/main/data-prep-kit/walmart-reports-1/Walmart_2024.pdf', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'Walmart_2024_copy.pdf' ))  # create a dupe file

Local file 'input/Walmart-10K-Reports-Optimized_2023.pdf' (1.61 MB) already exists. Skipping download.
Local file 'input/Walmart_2024.pdf' (4.87 MB) already exists. Skipping download.
Local file 'input/Walmart_2024_copy.pdf' (4.87 MB) already exists. Skipping download.


### Set input/output path variables for the pipeline

In [3]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_chunk_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_exact_dedupe_out')
output_docid_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_docid_out')
output_fuzzy_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '05_fuzzy_dedupe_out')
output_doc_quality_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '06_doc_quality_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '07_embeddings_out')

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


### Import Common python modules

In [4]:
import os
import sys

# Main repo root
from utils import rootdir

from data_processing.runtime.pure_python import PythonTransformLauncher
from data_processing.utils import ParamsUtils

<a id="pdf2parquet"></a>

## Step-2: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### Set Input/output Folder

In [5]:
STAGE = 1 

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  output_parquet_dir

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_parquet_out'


### Execute 

In [6]:
%%time 

import ast
import os
import sys

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration

from data_processing.utils import GB, ParamsUtils


# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = PythonTransformLauncher(Pdf2ParquetPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Job failed")


23:52:33 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': True, 'double_precision': 8}
23:52:33 INFO - pipeline id pipeline_id
23:52:33 INFO - code location None
23:52:33 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_parquet_out
23:52:33 INFO - data factory data_ max_files -1, n_sample -1
23:52:33 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
23:52:33 INFO - orchestrator pdf2parquet started at 2024-09-15 23:52:33
23:52:33 INFO - Number of files is 3, source profile {'max_file_size': 4.640201568603516, 'min_file_size': 1.5370569229125977, 'total_file_size': 10.817460060119629}
23:52:33 INFO - Initializing models


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

2024-09-15 23:54:23.623 ( 112.335s) [        F132D740]    doc_normalisation.h:448   WARN| found new `other` type: checkbox-unselected
2024-09-15 23:54:23.623 ( 112.336s) [        F132D740]    doc_normalisation.h:448   WARN| found new `other` type: checkbox-selected
2024-09-15 23:54:23.623 ( 112.336s) [        F132D740]    doc_normalisation.h:448   WARN| found new `other` type: checkbox-unselected
2024-09-15 23:54:23.623 ( 112.336s) [        F132D740]    doc_normalisation.h:448   WARN| found new `other` type: checkbox-unselected
2024-09-15 23:54:23.623 ( 112.336s) [        F132D740]    doc_normalisation.h:448   WARN| found new `other` type: checkbox-selected
23:54:23 INFO - Completed 1 files (33.33%) in 1.778 min
2024-09-15 23:56:08.187 ( 216.900s) [        F132D740]    doc_normalisation.h:448   WARN| found new `other` type: checkbox-unselected
23:56:08 INFO - Completed 2 files (66.67%) in 3.527 min
2024-09-15 23:57:53.054 ( 321.767s) [        F132D740]    doc_normalisation.h:448   WARN

✅ Stage:1 completed successfully
CPU times: user 13min 44s, sys: 3.62 s, total: 13min 47s
Wall time: 5min 22s


### Inspect Generated output

Here we should see one entry per input file processed

In [7]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (3, 12)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,Walmart-10K-Reports-Optimized_2023.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",100,82,1158,63d14368-4cbc-46e7-aede-f103cfdc5b5b,pdf,49138b69e8da8581e155885fb487ff96e9fe2fa24e7e6c...,1255781,2024-09-15T23:54:23.967489,106.661048,Walmart-10K-Reports-Optimized_2023.pdf
1,Walmart_2024_copy.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",100,83,1058,5c1590b9-c09f-4842-8a67-02bbaf73e5df,pdf,4f8daeb70531619b07ca64624526e7785e802643c653ea...,1215221,2024-09-15T23:57:53.629710,104.700134,Walmart_2024_copy.pdf
2,Walmart_2024.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",100,83,1058,d0f0c84e-8b9f-4c01-a1a5-65d3c675400e,pdf,29190b8923fb2153321dc52292e19b359a96c2e9974048...,1215216,2024-09-15T23:56:08.895449,104.766304,Walmart_2024.pdf


<a id="chunking"></a>

##  Step-3: Doc chunks

Split the documents in chunks, according to their layout segmentation.

### Set Input/output Folder

In [8]:
STAGE = 2

input_folder = output_parquet_dir # previous output folder is the input folder for the current stage
output_folder =  output_chunk_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_chunk_out'


### Execute 

In [9]:
%%time 

# Import doc_json_chunk transform configuration
from doc_chunk_transform_python import DocChunkPythonTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = PythonTransformLauncher(DocChunkPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Job failed")

23:57:54 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'output_chunk_column_name': 'contents', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox'}
23:57:54 INFO - pipeline id pipeline_id
23:57:54 INFO - code location None
23:57:54 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_chunk_out
23:57:54 INFO - data factory data_ max_files -1, n_sample -1
23:57:54 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:57:54 INFO - orchestrator doc_chunk started at 2024-09-15 23:57:54
23:57:54 INFO - Number of files is 3, source profile {'max_file_size': 0.23883724212646484, 'min_file_size': 0.20527362823486328, 'total_file_size': 0.6494197845458984}
23:57:54 INFO - Comp

✅ Stage:2 completed successfully
CPU times: user 1.46 s, sys: 86.1 ms, total: 1.55 s
Wall time: 1.54 s


### Inspect Generated output

We would see documents are split into many chunks

In [10]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Files processed : 3
Chunks created : 1,973
Input data dimensions (rows x columns)=  (3, 12)
Output data dimensions (rows x columns)=  (1973, 15)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox
1090,Walmart_2024_copy.pdf,100,83,1058,5c1590b9-c09f-4842-8a67-02bbaf73e5df,pdf,4f8daeb70531619b07ca64624526e7785e802643c653ea...,1215221,2024-09-15T23:57:53.629710,104.700134,Walmart_2024_copy.pdf,% @ E 6 % 6 E ? 4 @ > 6 ' 6 C @ > > @ ? * 9 2 ...,$.main-text[693],67,"[35.41247559, 377.0, 549.92785645, 438.14001465]"
1638,Walmart_2024.pdf,100,83,1058,d0f0c84e-8b9f-4c01-a1a5-65d3c675400e,pdf,29190b8923fb2153321dc52292e19b359a96c2e9974048...,1215216,2024-09-15T23:56:08.895449,104.766304,Walmart_2024.pdf,"3 1 4 & 2 = ! - & 6 * * 4 9 6 ( - & 7 * 6 3 , ...",$.main-text[503],48,"[35.38315201, 213.5, 534.5958252, 238.64001465]"
1955,Walmart_2024.pdf,100,83,1058,d0f0c84e-8b9f-4c01-a1a5-65d3c675400e,pdf,29190b8923fb2153321dc52292e19b359a96c2e9974048...,1215216,2024-09-15T23:56:08.895449,104.766304,Walmart_2024.pdf,"/ ) . "" 6 * % "" * . & 9 $ & 2 4 * ' 9 4 ) "" 4\...",$.main-text[1011],98,"[175.83938599, 391.23840332, 409.16744995, 407..."


## Step-4: Exact Dedup

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### Set Input/output Folder

In [11]:
STAGE  = 3

input_folder = output_chunk_dir # previous output folder is the input folder for the current stage
output_folder =  output_exact_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output/02_chunk_out' --> output='output/03_exact_dedupe_out'


### Execute 

In [12]:
%%time

# Import ededup transform configuration
from ededup_transform_python import EdedupPythonTransformRuntimeConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # ededup parameters
    "ededup_doc_column": "contents",
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = PythonTransformLauncher(EdedupPythonTransformRuntimeConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:57:55 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'document_id', 'use_snapshot': False, 'snapshot_directory': None}
23:57:55 INFO - pipeline id pipeline_id
23:57:55 INFO - code location None
23:57:55 INFO - data factory data_ is using local data access: input_folder - output/02_chunk_out output_folder - output/03_exact_dedupe_out
23:57:55 INFO - data factory data_ max_files -1, n_sample -1
23:57:55 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:57:55 INFO - orchestrator ededup started at 2024-09-15 23:57:55
23:57:55 INFO - Number of files is 3, source profile {'max_file_size': 0.1514434814453125, 'min_file_size': 0.1461324691772461, 'total_file_size': 0.44895172119140625}
23:57:55 INFO - Starting from the beginning
23:57:55 INFO - Completed 1 files (33.33%) in 0.0 min
23:57:55 INFO - Completed 2 files (66.67%) in 0.0 min
23:57:55 INFO - C

✅ Stage:3 completed successfully
CPU times: user 46.8 ms, sys: 6.02 ms, total: 52.8 ms
Wall time: 47.1 ms


### Inspect Generated output

In [13]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (1973, 15)
Output data dimensions (rows x columns)=  (2, 16)
Input chunks before exact dedupe : 1,973
Output chunks after exact dedupe : 2
Duplicate chunks removed :   1971


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,removed
0,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,63d14368-4cbc-46e7-aede-f103cfdc5b5b,pdf,49138b69e8da8581e155885fb487ff96e9fe2fa24e7e6c...,1255781,2024-09-15T23:54:23.967489,106.661048,Walmart-10K-Reports-Optimized_2023.pdf,"A message from our CEO\n""We started this new y...",$.main-text[6],2,"[29.09381294, 166.47808838, 500.76074219, 270....","[63d14368-4cbc-46e7-aede-f103cfdc5b5b, 63d1436..."
1,Walmart_2024.pdf,100,83,1058,d0f0c84e-8b9f-4c01-a1a5-65d3c675400e,pdf,29190b8923fb2153321dc52292e19b359a96c2e9974048...,1215216,2024-09-15T23:56:08.895449,104.766304,Walmart_2024.pdf,"A message from our CEO\n""At Walmart, we're a p...",$.main-text[6],2,"[30.18040848, 338.63272095, 526.46081543, 436....","[d0f0c84e-8b9f-4c01-a1a5-65d3c675400e, d0f0c84..."


## Step-5:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is a pre-requisite for fuzzy dedup** in the pipeline.

In [14]:

# Input for this stage is the output of exact dedeup component
# output of this component makes it possible for fdedup component to run on data.

STAGE  = 4

input_folder = output_exact_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_docid_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-4: Processing input='output/03_exact_dedupe_out' --> output='output/04_docid_out'


In [15]:
%%time 

from doc_id_transform_python import DocIDPythonTransformRuntimeConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "hash_column",
    "doc_id_int_column": "int_id_column",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = PythonTransformLauncher(DocIDPythonTransformRuntimeConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:57:55 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'hash_column', 'int_column': 'int_id_column', 'start_id': 0}
23:57:55 INFO - pipeline id pipeline_id
23:57:55 INFO - code location None
23:57:55 INFO - data factory data_ is using local data access: input_folder - output/03_exact_dedupe_out output_folder - output/04_docid_out
23:57:55 INFO - data factory data_ max_files -1, n_sample -1
23:57:55 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:57:55 INFO - orchestrator doc_id started at 2024-09-15 23:57:55
23:57:55 INFO - Number of files is 3, source profile {'max_file_size': 0.010014533996582031, 'min_file_size': 0.0033016204833984375, 'total_file_size': 0.02299976348876953}
23:57:55 INFO - Completed 1 files (33.33%) in 0.0 min
23:57:55 INFO - Completed 2 files (66.67%) in 0.0 min
23:57:55 WARNING - table is empty, skipping processing
23:57:5

✅ Stage:4 completed successfully
CPU times: user 19.6 ms, sys: 3 ms, total: 22.6 ms
Wall time: 18.8 ms


### Inspect Generated output

In [16]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (2, 16)
Output data dimensions (rows x columns)=  (2, 18)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,removed,hash_column,int_id_column
1,Walmart_2024.pdf,100,83,1058,d0f0c84e-8b9f-4c01-a1a5-65d3c675400e,pdf,29190b8923fb2153321dc52292e19b359a96c2e9974048...,1215216,2024-09-15T23:56:08.895449,104.766304,Walmart_2024.pdf,"A message from our CEO\n""At Walmart, we're a p...",$.main-text[6],2,"[30.18040848, 338.63272095, 526.46081543, 436....","[d0f0c84e-8b9f-4c01-a1a5-65d3c675400e, d0f0c84...",01b953276ba54dfa57da3f73dfcbbd26e62b634e8a3a41...,1
0,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,63d14368-4cbc-46e7-aede-f103cfdc5b5b,pdf,49138b69e8da8581e155885fb487ff96e9fe2fa24e7e6c...,1255781,2024-09-15T23:54:23.967489,106.661048,Walmart-10K-Reports-Optimized_2023.pdf,"A message from our CEO\n""We started this new y...",$.main-text[6],2,"[29.09381294, 166.47808838, 500.76074219, 270....","[63d14368-4cbc-46e7-aede-f103cfdc5b5b, 63d1436...",a651a5dc74053a1d5c92d134baf9ea23b7c009036a2619...,0


## Step-6: Fuzzy Dedup

**Fuzzy dedupe is currently available in RAY version**

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### Set Input/output Folder

In [17]:
## Input to this component is the output of doc_id generator component. 

STAGE  = 5

input_folder = output_docid_dir # previous output folder is the input folder for the current stage
output_folder =  output_fuzzy_dedupe_dir
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-5: Processing input='output/04_docid_out' --> output='output/05_fuzzy_dedupe_out'


### Execute 

In [18]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration
from data_processing_ray.runtime.ray import RayTransformLauncher


# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "int_id_column",
    "fdedup_cluster_column": "hash_column",
    # infrastructure
    "fdedup_bucket_cpu": 0.3,
    "fdedup_doc_cpu": 0.3,
    "fdedup_mhash_cpu": 0.3,
    "fdedup_num_doc_actors": 1,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 1,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.8,
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

23:57:55 INFO - Running locally
23:57:55 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'int_id_column', 'cluster_column': 'hash_column', 'bucket_cpu': 0.3, 'mhash_cpu': 0.3, 'doc_cpu': 0.3, 'num_doc_actors': 1, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 1, 'num_permutations': 64, 'threshold': 0.8, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 1}}
23:57:55 INFO - data factory data_ is using local data access: input_folder - output/04_docid_out output_folder - output/05_fuzzy_dedupe_out
23:57:55 INFO - data factory data_ max_files -1, n_sample -1
23:57:55 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:57:55 INFO - pipeline id pipeline_id
23:57:55 INFO - code location None
23:57:55 INFO - number of

✅ Stage:5 completed successfully
CPU times: user 511 ms, sys: 392 ms, total: 904 ms
Wall time: 34 s


### Inspect Generated output

In [19]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (2, 16)
Output data dimensions (rows x columns)=  (2, 18)
Duplicate chunks removed  by fuzzy-dedupe:   0


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename,contents,doc_jsonpath,page_number,bbox,removed,int_id_column,hash_column
0,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,63d14368-4cbc-46e7-aede-f103cfdc5b5b,pdf,49138b69e8da8581e155885fb487ff96e9fe2fa24e7e6c...,1255781,2024-09-15T23:54:23.967489,106.661048,Walmart-10K-Reports-Optimized_2023.pdf,"A message from our CEO\n""We started this new y...",$.main-text[6],2,"[29.09381294, 166.47808838, 500.76074219, 270....","[63d14368-4cbc-46e7-aede-f103cfdc5b5b, 63d1436...",0,-1
1,Walmart_2024.pdf,100,83,1058,d0f0c84e-8b9f-4c01-a1a5-65d3c675400e,pdf,29190b8923fb2153321dc52292e19b359a96c2e9974048...,1215216,2024-09-15T23:56:08.895449,104.766304,Walmart_2024.pdf,"A message from our CEO\n""At Walmart, we're a p...",$.main-text[6],2,"[30.18040848, 338.63272095, 526.46081543, 436....","[d0f0c84e-8b9f-4c01-a1a5-65d3c675400e, d0f0c84...",1,-1


## Step-7: Document Quality

### Set Input/output Folder

In [20]:
STAGE  = 6

input_folder = output_fuzzy_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_doc_quality_dir
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-6: Processing input='output/05_fuzzy_dedupe_out' --> output='output/06_doc_quality_out'


### Execute 

In [21]:
%%time

import os
import sys
from pathlib import Path

from doc_quality_transform import (
    text_lang_cli_param,
    doc_content_column_cli_param,
    bad_word_filepath_cli_param,
)
from doc_quality_transform_python import DocQualityPythonTransformConfiguration
from data_processing.utils import ParamsUtils

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}

doc_quality_basedir = os.path.join(rootdir, "transforms", "language", "doc_quality", "python")
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # doc quality configuration
    text_lang_cli_param: "en",
    doc_content_column_cli_param: "contents",
    bad_word_filepath_cli_param: os.path.join(doc_quality_basedir, "ldnoobw", "en"),
}


Path(output_folder).mkdir(parents=True, exist_ok=True)

sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = PythonTransformLauncher(DocQualityPythonTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Job failed")

23:58:29 INFO - doc_quality parameters are : {'text_lang': 'en', 'doc_content_column': 'contents', 'bad_word_filepath': '/home/sujee/my-stuff/projects/ai-alliance/data-prep-kit-sujee-dev/transforms/language/doc_quality/python/ldnoobw/en', 's3_cred': None, 'docq_data_factory': <data_processing.data_access.data_access_factory.DataAccessFactory object at 0x7092cf227490>}
23:58:29 INFO - data factory docq_ is using local configuration without input/output path
23:58:29 INFO - data factory docq_ max_files -1, n_sample -1
23:58:29 INFO - data factory docq_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:58:29 INFO - pipeline id pipeline_id
23:58:29 INFO - code location None
23:58:29 INFO - data factory data_ is using local data access: input_folder - output/05_fuzzy_dedupe_out output_folder - output/06_doc_quality_out
23:58:29 INFO - data factory data_ max_files -1, n_sample -1
23:58:29 INFO - data fact

✅ Stage:6 completed successfully
CPU times: user 30.8 ms, sys: 3.01 ms, total: 33.8 ms
Wall time: 29.5 ms


### Inspect Generated output

In [22]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (2, 16)
Output data dimensions (rows x columns)=  (2, 29)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,...,docq_mean_word_len,docq_symbol_to_word_ratio,docq_sentence_count,docq_lorem_ipsum_ratio,docq_curly_bracket_ratio,docq_contain_bad_word,docq_bullet_point_ratio,docq_ellipsis_line_ratio,docq_alphabet_word_ratio,docq_contain_common_en_words
0,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,63d14368-4cbc-46e7-aede-f103cfdc5b5b,pdf,49138b69e8da8581e155885fb487ff96e9fe2fa24e7e6c...,1255781,2024-09-15T23:54:23.967489,106.661048,...,4.83871,0.0,3,0.0,0.0,False,0.0,0.0,1.0,True
1,Walmart_2024.pdf,100,83,1058,d0f0c84e-8b9f-4c01-a1a5-65d3c675400e,pdf,29190b8923fb2153321dc52292e19b359a96c2e9974048...,1215216,2024-09-15T23:56:08.895449,104.766304,...,5.16000,0.0,4,0.0,0.0,False,0.0,0.0,1.0,True


## Step-8:   Text encoding

Encode text for the vector storage.

In [23]:
STAGE  = 7

input_folder = output_doc_quality_dir
output_folder =  output_embeddings_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-7: Processing input='output/06_doc_quality_out' --> output='output/07_embeddings_out'


In [24]:
%%time 

from text_encoder_transform_python import TextEncoderPythonTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = PythonTransformLauncher(TextEncoderPythonTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Job failed")

23:58:30 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
23:58:30 INFO - pipeline id pipeline_id
23:58:30 INFO - code location None
23:58:30 INFO - data factory data_ is using local data access: input_folder - output/06_doc_quality_out output_folder - output/07_embeddings_out
23:58:30 INFO - data factory data_ max_files -1, n_sample -1
23:58:30 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:58:30 INFO - orchestrator text_encoder started at 2024-09-15 23:58:30
23:58:30 INFO - Number of files is 2, source profile {'max_file_size': 0.015343666076660156, 'min_file_size': 0.01500701904296875, 'total_file_size': 0.030350685119628906}
23:58:31 INFO - Completed 1 files (50.0%) in 0.001 min
23:58:31 INFO - Completed 2 files (100.0%) in 0.001 min
23:58:31

✅ Stage:7 completed successfully
CPU times: user 491 ms, sys: 151 ms, total: 642 ms
Wall time: 2.18 s


### Inspect Generated output

In [25]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (2, 29)
Output data dimensions (rows x columns)=  (2, 30)


,filename,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,...,docq_symbol_to_word_ratio,docq_sentence_count,docq_lorem_ipsum_ratio,docq_curly_bracket_ratio,docq_contain_bad_word,docq_bullet_point_ratio,docq_ellipsis_line_ratio,docq_alphabet_word_ratio,docq_contain_common_en_words,embeddings
1,Walmart_2024.pdf,100,83,1058,d0f0c84e-8b9f-4c01-a1a5-65d3c675400e,pdf,29190b8923fb2153321dc52292e19b359a96c2e9974048...,1215216,2024-09-15T23:56:08.895449,104.766304,...,0.0,4,0.0,0.0,False,0.0,0.0,1.0,True,"[0.007453779, 0.08018814, 0.051178806, 0.00213..."
0,Walmart-10K-Reports-Optimized_2023.pdf,100,82,1158,63d14368-4cbc-46e7-aede-f103cfdc5b5b,pdf,49138b69e8da8581e155885fb487ff96e9fe2fa24e7e6c...,1255781,2024-09-15T23:54:23.967489,106.661048,...,0.0,3,0.0,0.0,False,0.0,0.0,1.0,True,"[0.014166234, 0.07892205, 0.068165675, -0.0189..."


## Step-9: Copy output to final output dir

In [26]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/07_embeddings_out' --> 'output/output_final'
